# Data Augmentation for certain amount


v1.0.3

If data less than x, use data augmentation

In [1]:
import os
import numpy as np
from tqdm.notebook import tqdm
from keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img, save_img

Using TensorFlow backend.


In [2]:
seed_value = 1000
batchs_size = 1

data_type = "test"
char_size = 24

if data_type=="train":
    char_limit = 400
elif data_type=="val":
    char_limit = 50
elif data_type=="test":
    char_limit = 100
else:
    char_limit = None

class DataAugmentation:
    def __init__(self):
        self._path_data = "../../../extraction/2-release/80-20-ext/" + data_type + "/char-" + str(char_size)
        self._save_path = data_type + "-" + str(char_size) + "-" + str(char_limit)
        
        if not os.path.exists(self._save_path):
            os.mkdir(self._save_path)
        
    def is_augmenting(self):
        """ Another Parameter
        #height_shift_range = 0.1,
        #shear_range = 0.1,
        #zoom_range = 0.1,
        #width_shift_range = 0.1,
        """
        
        randState = np.random.RandomState(seed_value)
        
        datagen = ImageDataGenerator(
            rotation_range = 10,
            shear_range = 0.1,
            brightness_range = (0.5, 1.5), # 1 is neutral
            fill_mode = 'nearest')
        
        classes = sorted(os.listdir(self._path_data))
        
        for class_char in tqdm(classes):
            path_classes = os.path.join(self._path_data, class_char)
            path_save = os.path.join(self._save_path, class_char)
            
            if not os.path.exists(path_save):
                os.mkdir(path_save)
            
            images = []
            for img_name in os.listdir(path_classes):
                loaded_img = load_img(os.path.join(path_classes, img_name), color_mode='grayscale')
                img_array = img_to_array(loaded_img)
                images.append(img_array)
            
            if len(images) > char_limit:
                images = randState.permutation(images)
                images = images[:char_limit]
                i = 1
                for img in images:
                    save_img(os.path.join(path_save, str(i)+".jpg"), img)
                    i += 1
            elif len(images) == char_limit:
                i = 1
                for img in images:
                    save_img(os.path.join(path_save, str(i)+".jpg"), img)
                    i += 1
            else:
                images = np.array(images).reshape((-1,) + images[0].shape) # 4 DImension
                
                i = 1
                for batch in datagen.flow(images, batch_size=batchs_size,
                                          save_to_dir=path_save, save_prefix=class_char, save_format='jpg'):
                    i += 1
                    
                    if i > char_limit:
                        break
        
        print('augmentation data succeed')
                
if __name__=='__main__':
    da = DataAugmentation()
    da.is_augmenting()


augmentation data succeed
